In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import ast

from fastDamerauLevenshtein import damerauLevenshtein
from opportunistic_planning import data, sequence
from imp import reload
from scipy.stats import friedmanchisquare, wilcoxon

In [4]:
# read data

df = pd.read_csv('all_task_environments_2020-12-07.csv', header=0)

In [5]:
# dict generation ~1 sec

%time distances_dict = data.generate_distances_dict(df)

CPU times: user 1.06 s, sys: 12.1 ms, total: 1.08 s
Wall time: 1.07 s


In [6]:
# n=1: ~2min 50
# n=10: ~9min 35
# n=100: 1h 13min
%time results_new = data.calculate_edit_distances(df, distances_dict=distances_dict, n=100)

CPU times: user 1h 19min 3s, sys: 2.56 s, total: 1h 19min 5s
Wall time: 1h 19min 6s


In [7]:
results_new

c: 1.0; k: 0.0,0.1,1.1; x  c: 1.0; k: 0.0,0.1,1.1; y  \
0                     0.666667                   0.833333   
1                     0.750000                   0.875000   
2                     0.444444                   0.666667   
3                     0.538462                   0.692308   
4                     0.888889                   0.777778   
..                         ...                        ...   
175                   0.666667                   0.500000   
176                   0.500000                   0.500000   
177                   0.500000                   0.333333   
178                   0.500000                   0.333333   
179                   0.500000                   0.500000   

     c: 1.0; k: 0.0,0.1,1.1; z  c: 1.0; k: 0.0,0.1,1.1; xy  \
0                     0.833333                    0.833333   
1                     0.875000                    0.875000   
2                     0.555556                    0.555556   
3                     0.692308                    0.615385   
4                     0.888889                    0.666667   
..                         ...                         ...   
175                   0.666667                    0.500000   
176                   0.666667                    0.666667   
177                   0.500000                    0.333333   
178                   0.500000                    0.500000   
179                   0.666667                    0.666667   

     c: 1.0; k: 0.0,0.1,1.1; xz  c: 1.0; k: 0.0,0.1,1.1; yz  \
0                      0.833333                    0.833333   
1                      0.875000                    0.875000   
2                      0.444444                    0.666667   
3                      0.615385                    0.692308   
4                      0.888889                    0.777778   
..                          ...                         ...   
175                    0.500000                    0.500000   
176                    0.500000                    0.500000   
177                    0.333333                    0.500000   
178                    0.333333                    0.333333   
179                    0.500000                    0.500000   

     c: 1.0; k: 0.0,0.1,1.1; xyz  c: 1.1; k: 0.0,0.1,1.1; x  \
0                       0.833333                   0.666667   
1                       0.875000                   0.750000   
2                       0.555556                   0.444444   
3                       0.615385                   0.538462   
4                       0.666667                   0.888889   
..                           ...                        ...   
175                     0.666667                   0.666667   
176                     0.666667                   0.500000   
177                     0.333333                   0.500000   
178                     0.333333                   0.500000   
179                     0.666667                   0.500000   

     c: 1.1; k: 0.0,0.1,1.1; y  c: 1.1; k: 0.0,0.1,1.1; z  ...  \
0                     0.833333                   0.833333  ...   
1                     0.875000                   0.875000  ...   
2                     0.666667                   0.555556  ...   
3                     0.692308                   0.692308  ...   
4                     0.777778                   0.888889  ...   
..                         ...                        ...  ...   
175                   0.500000                   0.666667  ...   
176                   0.500000                   0.666667  ...   
177                   0.500000                   0.500000  ...   
178                   0.333333                   0.500000  ...   
179                   0.500000                   0.666667  ...   

     c: 1.9; k: 0.8,0.9,1.9; x  c: 1.9; k: 0.8,0.9,1.9; y  \
0                     0.666667                   0.833333   
1                     0.750000                   0.750000   
2                     0.444444       

In [9]:
#results_new100.T.to_csv('results/results_all_n100_2020-12-09.csv', header=True, index=True)

In [8]:
lowest, lowest_idx, lowest_median, list_mean, results_mean = data.get_lowest_error(results_new)
lowest, lowest_idx, lowest_median, np.nanmean(results_new['error'])

(0.6013803598362422,
 Index(['c: 1.5; k: 0.2,0.3,1.9; xyz'], dtype='object'),
 0.625,
 0.7582944444444444)

### Plot dimensions

In [18]:
# Define input for plots

c = [float(x[3:6]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'ID' and x != 'error']
k = [float(x[11:14]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'ID' and x != 'error']
dim = [x.strip() for x in results_new.columns[:-3].str.split(';').str[2]]
median = [x for x in results_mean.loc['mean'].tolist() if pd.notnull(x)]

In [19]:
%matplotlib qt

#cm = ['red','blue','green','magenta','cyan','orange','grey']
cm = ['#e74c3c', '#0064a3', '#70b85d', '#287d78', '#54d0ff', '#f1e664', '#fd8f00']
dim_num = [0 if x=='x' else 1 if x=='y' else 2 if x=='z' else 3 if x=='xy' else 4 if x=='xz' else 5 if x=='yz' else 6 for x in dim]

cmap = matplotlib.colors.ListedColormap(cm)

ticks = ['x', 'y', 'z', 'xy', 'xz', 'yz', 'xyz']
norm = matplotlib.colors.BoundaryNorm(ticks, cmap.N)

# create figure, 3d grid, set background to white
fig2 = plt.figure(figsize=(12,8))
ax2 = fig2.add_subplot(111, projection='3d')
ax2.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))

# create plot w/ median edit distance
img = ax2.scatter(c, k, median, alpha=0.5, s=38, c=dim_num, cmap=cmap)

# plot horizontal plane for baseline
xx, yy = np.meshgrid(np.linspace(1,1.9), np.linspace(0,0.9))
#yy = np.meshgrid(range(2), range(2))
zz = xx * 0 + np.mean(results_mean['error'])
ax2.plot_surface(xx, yy, zz, alpha=0.5)

# set labels
ax2.set_ylabel('parameter k', fontsize=20, labelpad=10)
ax2.set_xlabel('parameter c', fontsize=20, labelpad=10)
ax2.set_zlabel('normalized edit distance', fontsize=20, labelpad=7)
#plt.title('Average edit distance (1 step)', fontsize=24)

# create colorbar
cb = plt.colorbar(img, cax = fig2.add_axes([0.82,0.3,0.03,0.4]))
cb.set_alpha(0.8)
cb.draw_all()
cb.ax.set_yticklabels(ticks, fontsize=16)
plt.show()

#plt.gca().xaxis.set_major_locator(plt.NullLocator())
#plt.gca().yaxis.set_major_locator(plt.NullLocator())
#plt.savefig('figures/test.svg', format='svg', bbox_inches='tight', pad_inches = 0)

/home/zauri/files/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: FixedFormatter should only be used together with FixedLocator


### Plot best model with baselines

In [20]:
#%matplotlib inline
%matplotlib qt

IDs = df['ID']

error = df['error']

seqs= results_new['sequence'][:-2].values

res = results_new['c: 1.5; k: 0.2,0.3,1.9; xyz'][:-2].values
median = [np.median(res)] * len(res)

x = [x for x in range (0,len(seqs))]

cpt_median = [0.33333333333333337 for x in range(0,len(seqs))]

# plot scatter + lines for all simulations
plt.scatter(x, res, marker='o', s=20, c='blue', alpha=0.6, 
            label=str('model-generated median: ') + str(round(lowest_median,2)))
#plt.plot(x, res, c='blue', alpha=0.6)
plt.plot(x, median, c='blue', alpha=0.6)

# plot CPT baseline
plt.scatter(x, cpt_median, marker='o', s=20, c='darkgreen', alpha=0.3, 
            label=str('CPT baseline median: ') + str(round(np.median(cpt_median),2)))
plt.plot(x, [np.median(cpt_median)] * len(x), '-', c='darkgreen', alpha=0.6)
#plt.plot((x_original,x_original),(res_original,distances), '--', c='darkgreen', alpha=0.6)
plt.fill_between(x, cpt_median, alpha=0.2, color='darkgreen')

# plot random baseline
plt.scatter(x, error, c='red', s=20, marker='o', alpha=0.3, 
            label=str('guessing baseline median: ') + str(round(np.nanmedian(results_new['error']),2)))
plt.plot(x, [np.median(error)] * len(x), c='red', alpha=0.6)#, 
         #label=str('guessing baseline avg.: ' + str(round(np.mean(results_original['error']),2))))
plt.fill_between(x, error, [1.0 * len(error)], color='red', alpha=0.2)


plt.xticks(x, labels=IDs, rotation=90, fontsize=5)
#plt.xticklabels(IDs, rotation=90, fontsize=6)

plt.ylabel('edit distance', fontsize=10)
plt.xlabel('sequence', fontsize=10)
plt.ylim(0.0, 1.0)
plt.title('best model (all), c:1.5; k: 0.2,0.3,1.9; xyz')

plt.legend(fontsize=8, framealpha=0.9, loc='lower left')

#plt.savefig('plot_median_editdist_individualerrors_diff.png', bbox_inches='tight')
plt.show()

### Prep data for stat analysis

In [13]:
# Compare avg edit distance for xy and xyz
list_xy = []
list_xyz = []

for col in results_mean:
    if col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xy':
        list_xy.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xyz':
        list_xyz.append(results_mean.at['mean',col])
        
avg_xy = np.mean(list_xy)
med_xy = np.median(list_xy)
std_xy = np.std(list_xy)
print('Average xy: ' + str(avg_xy)  + ', stdev: ' + str(std_xy) + ', median: ' + str(med_xy))
        
avg_xyz = np.mean(list_xyz)
med_xyz = np.median(list_xyz)
std_xyz = np.std(list_xyz)
print('Average xyz: ' + str(avg_xyz) + ', stdev: ' + str(std_xyz) + ', median: ' + str(med_xyz))

Average xy: 0.6386783556493221, stdev: 0.028177756564845304, median: 0.6321446379963537
Average xyz: 0.6369766233585981, stdev: 0.027652217382083854, median: 0.6268402163377654


In [14]:
# Compare avg edit distances for x, y, z
list_x = []
list_y = []
list_z = []
list_xz = []
list_yz = []

for col in results_mean:
    if col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'x':
        list_x.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'y':
        list_y.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'z':
        list_z.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xz':
        list_xz.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'yz':
        list_yz.append(results_mean.at['mean',col])

avg_xz = np.mean(list_xz)
avg_yz = np.mean(list_yz)
        
avg_x = np.mean(list_x)
lowest_x = min(list_x)
print('Average x: ' + str(avg_x) + ', min: ' + str(lowest_x)) 
        
avg_y = np.mean(list_y)
lowest_y = min(list_y)
print('Average y: ' + str(avg_y) + ', min: ' + str(lowest_y)) 
        
avg_z = np.mean(list_z)
lowest_z = min(list_z)
print('Average z: ' + str(avg_z) + ', min: ' + str(lowest_z)) 

print('Average xz: ' + str(avg_xz)) 
print('Average yz: ' + str(avg_yz)) 

Average x: 0.7011170578280843, min: 0.6817964869190358
Average y: 0.677292692876934, min: 0.642845792551675
Average z: 0.6983740347964656, min: 0.6678744730215318
Average xz: 0.6861727570302822
Average yz: 0.6762890034594675


In [15]:
# Compare all error predictions for dimensions
stat, p = friedmanchisquare(list_x,list_y,list_z,list_xy,list_xz,list_yz,list_xyz)
print('Friedman: stat = %.3f, p = % 10.3E' % (stat, p))

Friedman: stat = 2545.751, p =  0.000E+00


In [16]:
# 2D vs 3D median
stat, p = wilcoxon(list_xy, list_xyz, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 92688.500, p = 0.00000
